In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("postcode_insights")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/25 16:53:57 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/25 16:53:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 16:54:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/dwijeratne/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dwijeratne/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/

KeyboardInterrupt: 

23/09/25 16:57:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
postcode = spark.read.parquet("../../../data/insights/pre_insights/postcode.parquet/")

In [ ]:
postcode.columns

In [ ]:
postcode.show(truncate = False)

In [ ]:
postcode_agg = postcode.groupBy("merchant_abn").agg\
                (f.first("merchant_name").alias("name"),
                 f.countDistinct("consumer_postcode").alias("number_of_postcodes"),
                 f.mode("consumer_postcode").alias("most_popular_postcode"),
                 f.avg("total_weekly_personal_income").alias("avg_total_weekly_personal_income"),
                 f.avg("total_weekly_fam_income").alias("avg_total_weekly_fam_income"),
                 f.avg("median_age").alias("avg_median_age"),
                 f.avg("avg_household_size").alias("avg_household_size"))

In [ ]:
total_number_postcodes = postcode.select(f.col("consumer_postcode")).distinct().count()

In [ ]:
postcode_agg = postcode_agg.withColumn("postcode_reach", postcode_agg.number_of_postcodes/total_number_postcodes)

In [ ]:
postcode_agg.orderBy(f.col("number_of_postcodes").desc()).show(5,truncate = False)

Checking the proportion of the most popular postcode of the merchant whech served the highest number of consumers in different postcode, i.e, Erat Vitae LLP

In [ ]:
EVLLP = postcode.filter(postcode.merchant_abn == 24852446429)
mode_pc_EVLLP = EVLLP.filter(EVLLP.consumer_postcode == 6470).count()
total_pc_EVLLP = 241547
proportion = mode_pc_EVLLP/total_pc_EVLLP
print(proportion)

Using the most popular postcode seems infeasable as the proportion of the most popular postcode is very small.

In [ ]:
postcode_agg = postcode_agg.drop("most_popular_postcode")

In [ ]:
consumer_id_count_per_postcode = postcode.groupBy("merchant_abn","consumer_postcode").agg(f.count("consumer_id").alias("number_of_consumers"))
avg_num_of_consumers_per_postcode = consumer_id_count_per_postcode.groupBy("merchant_abn").agg(f.avg("number_of_consumers").alias("avg_num_of_consumers_per_postcode"))

In [ ]:
postcode_agg = postcode_agg.join(avg_num_of_consumers_per_postcode, on = "merchant_abn", how = "leftouter")

In [ ]:
postcode_agg.orderBy(f.col("avg_num_of_consumers_per_postcode").desc()).show(5,truncate = False)

In [ ]:
postcode.select("median_age").describe()

In [ ]:
postcode_agg.select("avg_median_age").describe()

In [ ]:
postcode_agg.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/postcode_agg.parquet")

In [ ]:
spark.stop()

### Insights into postcodes
- Most consumers are aged between 30-50 years old.
